In [1]:
import os
import subprocess
from osgeo import gdal

In [6]:
# Define the input GeoTIFF file, intermediate reprojected file, and output directory
input_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif'
reprojected_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif'
output_directory = '/Users/vivianhuang/Documents/GitHub/raster_process/final_data/'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Reproject the GeoTIFF to EPSG:3857
def reproject_geotiff(input_file, output_file, dst_srs='EPSG:3857', nodata_value=0):
    print(f"Reprojecting {input_file} to {output_file} with SRS {dst_srs}")
    input_ds = gdal.Open(input_file)
    if input_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")
    
    # Set the nodata value
    gdal.Warp(output_file, input_ds, dstSRS=dst_srs, srcNodata=nodata_value, dstNodata=nodata_value)
    print("GeoTIFF reprojected successfully")

# Generate tiles from the reprojected GeoTIFF
def generate_tiles(input_file, output_dir, zoom_levels):
    print(f"Generating tiles from {input_file} to {output_dir} for zoom levels {zoom_levels}")
    input_ds = gdal.Open(input_file)
    if input_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")

    xsize = input_ds.RasterXSize
    ysize = input_ds.RasterYSize
    print(f"Input dataset dimensions: xsize={xsize}, ysize={ysize}")
    
    tile_size = 256

    for zoom in range(zoom_levels[0], zoom_levels[1] + 1):
        scale = 2 ** zoom
        tile_grid_xsize = (xsize * scale) // tile_size
        tile_grid_ysize = (ysize * scale) // tile_size

        if tile_grid_xsize == 0 or tile_grid_ysize == 0:
            print(f"Skipping zoom level {zoom} due to zero-sized tile grid")
            continue

        print(f"Zoom level {zoom}: scale={scale}, tile_grid_xsize={tile_grid_xsize}, tile_grid_ysize={tile_grid_ysize}")

        for x in range(tile_grid_xsize):
            for y in range(tile_grid_ysize):
                xoff = x * tile_size // scale
                yoff = y * tile_size // scale
                win_xsize = min(tile_size // scale, xsize - xoff)
                win_ysize = min(tile_size // scale, ysize - yoff)

                tile_filename = os.path.join(output_dir, str(zoom), str(x), f"{y}.png")
                os.makedirs(os.path.dirname(tile_filename), exist_ok=True)

                print(f"Creating tile {tile_filename} from window ({xoff}, {yoff}, {win_xsize}, {win_ysize})")
                gdal.Translate(tile_filename, input_ds,
                               srcWin=[xoff, yoff, win_xsize, win_ysize],
                               width=tile_size, height=tile_size,
                               outputType=gdal.GDT_Byte,
                               format="PNG")

    print("Tiles generated successfully")

# Reproject the GeoTIFF
reproject_geotiff(input_geotiff, reprojected_geotiff, nodata_value=0)

# Generate tiles from the reprojected GeoTIFF
generate_tiles(reprojected_geotiff, output_directory, (0, 5))

Reprojecting /Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif with SRS EPSG:3857
GeoTIFF reprojected successfully
Generating tiles from /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif to /Users/vivianhuang/Documents/GitHub/raster_process/final_data/ for zoom levels (0, 5)
Input dataset dimensions: xsize=60, ysize=39
Skipping zoom level 0 due to zero-sized tile grid
Skipping zoom level 1 due to zero-sized tile grid
Skipping zoom level 2 due to zero-sized tile grid
Zoom level 3: scale=8, tile_grid_xsize=1, tile_grid_ysize=1
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/3/0/0.png from window (0, 0, 32, 32)
Zoom level 4: scale=16, tile_grid_xsize=3, tile_grid_ysize=2
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/4/0/0.png from window (0, 0, 16, 16)
Creating tile /Users/vivianhuang/Do